In [1]:
print("Importa librerie", end="", flush=True)
import tensorflow as tf
print(".", end="", flush=True)
from sklearn.metrics import f1_score
print(".", end="", flush=True)
import keras_tuner as kt
print(".", end="", flush=True)
import pandas as pd
print(".", end="", flush=True)
import yfinance as yf
print(".", end="", flush=True)
import numpy as np
print(".", end="", flush=True)
import funzioni as fx
print(".", end="", flush=True)
import pickle
print(".", end="", flush=True)
import gc
print(".", end="", flush=True)
gc.enable()
import os
print("=", end="\n", flush=True)
print("ok")

Importa librerie

2023-10-20 13:28:37.894825: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 13:28:38.510720: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-20 13:28:39.799079: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.2/lib64:
2023-10-20 13:28:39.799614: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

..Using TensorFlow backend
.......=
ok


In [2]:
print("Versione TensorFlow:", tf.__version__)
print("Versione CUDA:", tf.sysconfig.get_build_info()['cuda_version'])
print("Versione cuDNN:", tf.sysconfig.get_build_info()['cudnn_version'])

Versione TensorFlow: 2.10.0
Versione CUDA: 11.2
Versione cuDNN: 8


In [3]:
import psutil

def print_memory_usage():
    print(f"Memory usage: {psutil.virtual_memory().percent}%")

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

2023-10-20 13:28:43.160015: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-20 13:28:43.440733: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.2/lib64:
2023-10-20 13:28:43.440759: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
simbolo_test = "BTG"
simbolo_validazione = "DHT"
n_simboli_addestramento = "Tutti"
epochs=50
batch_size=1000
n_timesteps = 60 # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
giorni_previsione = 1 # n. barre nel futuro di cui si desidera prevedere il prezzo
set_file_x_y = f"_{n_simboli_addestramento}"
initial_lr = 0.001

features_prezzo = [
    "Close",
#    "EMA_5", 
    "EMA_20", 
    "EMA_50",
    "Open",  
    "High",
    "Low",
    "PSAR",
    "SUPERT", 
]

features_da_scalare_singolarmente = [
    "Volume",
    "ATR",
    "PSARaf",
    "ADX",
]

features_meno_piu = [
    "MACDh",    
    "AROONOSC",
    "TRIX",
    "TRIXs",
    "DM_OSC",
]

features_no_scala = [
    "SUPERTd",  
    "PSARr"
]

features_candele = [
#    "CDL_2CROWS", "CDL_3BLACKCROWS", "CDL_3INSIDE", "CDL_3LINESTRIKE", "CDL_3OUTSIDE", "CDL_3STARSINSOUTH", "CDL_3WHITESOLDIERS", "CDL_ABANDONEDBABY", "CDL_ADVANCEBLOCK", "CDL_BELTHOLD", "CDL_BREAKAWAY", "CDL_CLOSINGMARUBOZU", "CDL_CONCEALBABYSWALL", "CDL_COUNTERATTACK", "CDL_DARKCLOUDCOVER", "CDL_DOJI_10_0.1", "CDL_DOJISTAR", "CDL_DRAGONFLYDOJI", "CDL_ENGULFING", "CDL_EVENINGDOJISTAR", "CDL_EVENINGSTAR", "CDL_GAPSIDESIDEWHITE", "CDL_GRAVESTONEDOJI", "CDL_HAMMER", "CDL_HANGINGMAN", "CDL_HARAMI", "CDL_HARAMICROSS", "CDL_HIGHWAVE", "CDL_HIKKAKE", "CDL_HIKKAKEMOD", "CDL_HOMINGPIGEON", "CDL_IDENTICAL3CROWS", "CDL_INNECK", "CDL_INSIDE", "CDL_INVERTEDHAMMER", "CDL_KICKING", "CDL_KICKINGBYLENGTH", "CDL_LADDERBOTTOM", "CDL_LONGLEGGEDDOJI", "CDL_LONGLINE", "CDL_MARUBOZU", "CDL_MATCHINGLOW", "CDL_MATHOLD", "CDL_MORNINGDOJISTAR", "CDL_MORNINGSTAR", "CDL_ONNECK", "CDL_PIERCING", "CDL_RICKSHAWMAN", "CDL_RISEFALL3METHODS", "CDL_SEPARATINGLINES", "CDL_SHOOTINGSTAR", "CDL_SHORTLINE", "CDL_SPINNINGTOP", "CDL_STALLEDPATTERN", "CDL_STICKSANDWICH", "CDL_TAKURI", "CDL_TASUKIGAP", "CDL_THRUSTING", "CDL_TRISTAR", "CDL_UNIQUE3RIVER", "CDL_UPSIDEGAP2CROWS", "CDL_XSIDEGAP3METHODS",
]

elenco_targets = [
#    "EMA_5",
#    "EMA_20", 
#    "EMA_50",
    #"Open",
    #"High",
    #"Low",
    "Target"
]

col_features_prezzo = {col: idx for idx, col in enumerate(features_prezzo)}
col_features_da_scalare_singolarmente = {col: idx for idx, col in enumerate(features_da_scalare_singolarmente)}
col_features_meno_piu = {col: idx for idx, col in enumerate(features_meno_piu)}
col_features_no_scala = {col: idx for idx, col in enumerate(features_no_scala)}
col_features_candele = {col: idx for idx, col in enumerate(features_candele)}
col_targets = {col: idx for idx, col in enumerate(elenco_targets)}
n_features = len(col_features_prezzo) + len(col_features_da_scalare_singolarmente) + len(col_features_meno_piu) + len(col_features_no_scala) + len(col_features_candele) 
n_targets = len(col_targets) 

def set_di_tuning(lista_ticker):
    if os.path.exists("dati/indice.npy"):    
        inizio = np.load("dati/indice.npy")
        print(inizio)
    else:
        inizio = 0

    if os.path.exists(f"dati/X{set_file_x_y}.npy") and os.path.exists(f"dati/Y{set_file_x_y}.npy"):
        print("Caricamento X e Y")
        X = np.load(f'dati/X{set_file_x_y}.npy')
        Y = np.load(f'dati/Y{set_file_x_y}.npy')
    else:
        X = np.zeros((0, n_timesteps, n_features))
        #Y = np.zeros((0, giorni_previsione, n_targets)) #togliere in caso di classificazione binaria
        Y = np.zeros((0, 1)) #solo per classificazione binaria
    if inizio < n_simboli_addestramento:
        lista_x, lista_y = [], []
        for i_ticker in range (inizio, n_simboli_addestramento):
            nome_simbolo = lista_ticker["Ticker"].iloc[i_ticker]
            print(f"\033[48;5;42m{i_ticker+1} di {n_simboli_addestramento}: Ticker {nome_simbolo}\033[0m")
            print("Download dati ticker", flush=True)
            try:
                ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
                if ticker["Close"].iloc[-1] >= 1:
                    ticker.index = ticker.index.date
                    print("Calcolo indicatori ticker", flush=True)
                    ticker = fx.crea_indicatori(ticker)
                    ticker.dropna(axis=0, inplace=True)

                    print("Definizione features e target", flush=True)
                    _, X_train, Y_train, _ = fx.to_XY(ticker, features_prezzo, features_da_scalare_singolarmente, features_meno_piu, features_candele, features_no_scala, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)

                    print("Aggiunta dati a liste X e Y", flush=True)
                    lista_x.append(X_train)  
                    print(X_train.shape)      
                    lista_y.append(Y_train)   
                    print(Y_train.shape)
                    if ((i_ticker + 1) % 100 == 0):
                        print("Concatenamento su X, Y", flush=True)
                        X_arr = np.concatenate(lista_x, axis=0)
                        Y_arr = np.concatenate(lista_y, axis=0)
                        X = np.concatenate((X, X_arr), axis=0)
                        Y = np.concatenate((Y, Y_arr), axis=0)
                        print("Salvataggio X e Y su file", flush=True)
                        np.save(f'dati/X{set_file_x_y}', X)
                        np.save(f'dati/Y{set_file_x_y}', Y)
                        np.save("dati/indice", i_ticker)
                        lista_x, lista_y = [], []
                    gc.collect()
            except Exception as e:
                print(e)
                gc.collect()
                continue
                
        print("Concatenamento su X, Y", flush=True)
        X_arr = np.vstack(lista_x)
        Y_arr = np.vstack(lista_y)
        X = np.vstack((X, X_arr))
        Y = np.vstack((Y, Y_arr))
        print("Salvataggio X e Y su file", flush=True)
        np.save(f'dati/X{set_file_x_y}', X)
        np.save(f'dati/Y{set_file_x_y}', Y)
        np.save("dati/indice", i_ticker)
        lista_x, lista_y = [], []

        gc.collect()
    return X, Y
        
def addestramento(lista_ticker):
    model = tf.keras.models.load_model("tuning.keras")
    
    if os.path.exists("dati/indice_addestramento.npy"):    
        inizio = np.load("dati/indice_addestramento.npy")
    else:
        inizio = 0
    print("ok")
    X = np.zeros((0, n_timesteps, n_features))
    #Y = np.zeros((0, giorni_previsione, n_targets)) #togliere in caso di classificazione binaria
    Y = np.zeros((0, 1)) #solo per classificazione binaria
    if inizio < n_simboli_addestramento:
        for i_ticker in range (inizio, n_simboli_addestramento):
            nome_simbolo = lista_ticker["Ticker"].iloc[i_ticker]
            print(f"\033[48;5;42m{i_ticker+1} di {n_simboli_addestramento}: Ticker {nome_simbolo}\033[0m")
            print("Download dati ticker", flush=True)
            try:
                ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
                if ticker["Close"].iloc[-1] >= 1:
                    ticker.index = ticker.index.date
                    print("Calcolo indicatori ticker", flush=True)
                    ticker = fx.crea_indicatori(ticker)
                    ticker.dropna(axis=0, inplace=True)

                    print("Definizione features e target", flush=True)
                    _, X_train, Y_train, _ = fx.to_XY(ticker, features_prezzo, features_da_scalare_singolarmente, features_meno_piu, features_candele, features_no_scala, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)

                    print(f"Addestramento simbolo {nome_simbolo}", flush=True)
                    model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val))
                    np.save("dati/indice_addestramento", i_ticker)
                    model.save('addestramento.keras')

                    gc.collect()
            except Exception as e:
                print(e)
                gc.collect()
                continue

def weighted_binary_crossentropy(pos_weight):
    def loss(y_true, y_pred):
        logloss = tf.keras.backend.mean(tf.keras.backend.binary_crossentropy(y_true, y_pred), axis=-1)
        return logloss * (1 + (pos_weight - 1) * y_true)
    return loss

def f1_score(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    true_positives = tf.cast(true_positives, dtype=tf.float64)
    predicted_positives = tf.cast(predicted_positives, dtype=tf.float64)
    possible_positives = tf.cast(possible_positives, dtype=tf.float64)
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    return f1_val

class MixedHyperModel(kt.engine.hypermodel.HyperModel):
    def __init__(self, n_timesteps, n_features, n_targets, giorni_previsione):
        self.n_timesteps = n_timesteps
        self.n_features = n_features
        self.n_targets = n_targets
        self.giorni_previsione = giorni_previsione

    def build(self, hp):
        model = tf.keras.models.Sequential()

        # Layer LSTM iniziale
        model.add(tf.keras.layers.LSTM(hp.Int('lstm_units_1', 50, 500, step=50),
                       input_shape=(self.n_timesteps, self.n_features), kernel_initializer='glorot_uniform'))
        model.add(tf.keras.layers.RepeatVector(self.giorni_previsione))

        # Aggiunta di layer LSTM intermedi
        for i in range(hp.Int('num_lstm_layers', 1, 4)):
            model.add(tf.keras.layers.LSTM(hp.Int(f'lstm_units_{i+2}', 50, 500, step=50), return_sequences=True,
                           kernel_regularizer=tf.keras.regularizers.l2(hp.Float(f'l2_rate_{i+2}', 0.000001, 5, step=0.001)), kernel_initializer='glorot_uniform'))
            model.add(tf.keras.layers.Dropout(hp.Float(f'lstm_dropout_{i+2}', 0, 0.5, step=0.1)))

        # Aggiunta di layer Dense
        for i in range(hp.Int('num_dense_layers', 1, 4)):
            model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(hp.Int(f'dense_units_{i}', 50, 500, step=50), activation='relu', kernel_initializer='glorot_uniform')))
            model.add(tf.keras.layers.Dropout(hp.Float(f'dense_dropout_{i}', 0, 0.5, step=0.1)))

        model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='glorot_uniform')))

        model.compile(optimizer="adam", 
              loss='binary_crossentropy', 
              metrics=['accuracy', 
                       tf.keras.metrics.Precision(name='precision'), 
                       tf.keras.metrics.Recall(name='recall'), 
                       tf.keras.metrics.AUC(name='auc')])
        
        return model

input_shape = (n_timesteps, n_features)
output_shape = (giorni_previsione, n_targets)
hypermodel = MixedHyperModel(n_timesteps, n_features, n_targets, giorni_previsione)

class F1Metric(tf.keras.callbacks.Callback):
    def __init__(self, validation_data): 
        self.validation_data = validation_data
    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_targ = self.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict)
        logs['val_f1'] = _val_f1

def custom_objective(trial):
    val_precision = trial.metrics.get('val_precision')
    return val_precision

def bay(hypermodel, X_train, Y_train, X_val, Y_val):
    bayesian_tuner = kt.tuners.BayesianOptimization(
        hypermodel,
        objective=objective,
        max_trials=50,
        directory='bayesian_search',
        project_name='bayesian_tuning'
    )
    
    bayesian_tuner.search_space_summary()    

    bayesian_tuner.search(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val), callbacks=callbacks)
    
    best_model_bayesian = bayesian_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_bayesian = bayesian_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Bayesian Optimization:", best_hyperparameters_bayesian.values)
    
    return bayesian_tuner, best_model_bayesian, best_hyperparameters_bayesian

def hb(hypermodel, X_train, Y_train, X_val, Y_val):
    hyperband_tuner = kt.tuners.Hyperband(
        hypermodel,
        objective='val_loss',
        max_trials=50,
        directory='hyperband_search',
        project_name='hyperband_tuning'
    )
    
    hyperband_tuner.search_space_summary()
    
    hyperband_tuner.search(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    
    best_model_hyperband = hyperband_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_hyperband = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Hyperband:", best_hyperparameters_hyperband.values)
    
    return best_model_hyperband, best_hyperparameters_hyperband

def rd(hypermodel, X_train, Y_train, X_val, Y_val):
    random_tuner = kt.tuners.RandomSearch(
        hypermodel,
        objective='val_loss',
        max_trials=50,
        executions_per_trial=2,
        directory='random_search',
        project_name='random_tuning'
    )
    random_tuner.search_space_summary()
    
    random_tuner.search(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    # Ottieni il miglior modello e i migliori iperparametri
    best_model_random = random_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_random = random_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Random Search:", best_hyperparameters_random.values)
    
    return best_model_random, best_hyperparameters_random

if not os.path.exists('dati'):
    os.makedirs('dati')

if os.path.exists("lista_ticker.parquet"):
    print("Caricamento lista_ticker esistente", flush=True)
    lista_ticker = pd.read_parquet("lista_ticker.parquet")
else:
    print("Download lista ticker", flush=True)
    lista_ticker = pd.read_parquet("Tickers_De_Giro.parquet")
    lista_ticker = lista_ticker.sample(frac=1).reset_index(drop=True)
    lista_ticker = lista_ticker.loc[(lista_ticker["Ticker"] != simbolo_test) & (lista_ticker["Ticker"] != simbolo_validazione) & (lista_ticker["Categoria"] != "D"), :]
    lista_ticker.to_parquet("lista_ticker.parquet")

if n_simboli_addestramento == 'Tutti': 
    n_simboli_addestramento = len(lista_ticker)

if os.path.exists("dati/ticker_val.parquet"):
    print("Caricamento dati ticker validazione", flush=True)
    ticker_val = pd.read_parquet("dati/ticker_val.parquet")
else:
    print("Download dati ticker validazione")
    ticker_val = yf.download(simbolo_validazione, start='2010-01-01', end='2023-12-31', progress=False)
    ticker_val.index = ticker_val.index.date
    print("Calcolo indicatori ticker validazione")
    ticker_val = fx.crea_indicatori(ticker_val)
    ticker_val.dropna(axis=0, inplace=True)
    ticker_val.to_parquet("dati/ticker_val.parquet")

if os.path.exists("dati/X_val.parquet") and os.path.exists("dati/Y_val.parquet"):
    print("Caricamento features e target ticker validazione", flush=True)
    X_val = np.load('dati/X_val.npy')
    Y_val = np.load('dati/Y_val.npy')
else:
    print("Definizione features e target ticker validazione", flush=True)
    _, X_val, Y_val, _ = fx.to_XY(ticker_val, features_prezzo, features_da_scalare_singolarmente, features_meno_piu, features_candele, features_no_scala, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)
    np.save(f'dati/X_val', X_val)
    np.save(f'dati/Y_val', Y_val)

objective = kt.Objective("val_precision", direction="max")

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
f1_metric = F1Metric(validation_data=(X_val, Y_val))
callbacks = [early_stopping, reduce_lr, f1_metric]

X, Y = set_di_tuning(lista_ticker)

print("Tuning")
print_memory_usage()
hypermodel = MixedHyperModel(n_timesteps, n_features, n_targets, giorni_previsione)
print_memory_usage()
model, par, bayesian_search = bay(hypermodel, X, Y, X_val, Y_val)
print_memory_usage()

trials = bayesian_search.oracle.trials

results = []

for trial in trials.values():
    trial_metrics = trial['metrics']
    val_f1 = trial_metrics['val_f1']['observations'][0]['value'] 
    results.append({
        'trial_id': trial['trial_id'],
        'val_f1': val_f1,  
        'hyperparameters': trial['hyperparameters'].values
    })

results_df = pd.DataFrame(results)
results_df.to_excel('results_df.xlsx')
    
model.save("tuning")
with open('hyperparameters.pkl', 'wb') as f:
    pickle.dump(par, f)

#addestramento(lista_ticker)
#    
print("ok")




Caricamento lista_ticker esistente
Caricamento dati ticker validazione
Definizione features e target ticker validazione
2580
Caricamento X e Y
Tuning
Memory usage: 80.8%
Memory usage: 80.8%


2023-10-20 13:28:52.997805: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Search space summary
Default search space size: 8
lstm_units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 500, 'step': 50, 'sampling': 'linear'}
num_lstm_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': 'linear'}
lstm_units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 500, 'step': 50, 'sampling': 'linear'}
l2_rate_2 (Float)
{'default': 1e-06, 'conditions': [], 'min_value': 1e-06, 'max_value': 5.0, 'step': 0.001, 'sampling': 'linear'}
lstm_dropout_2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
num_dense_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': 'linear'}
dense_units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 500, 'step': 50, 'sampling': 'linear'}
dense_dropout_0 (Float)
{'default': 0.0, 'conditions': [], 'min_value':

2023-10-20 13:28:57.667462: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 992575200 exceeds 10% of free system memory.


Epoch 1/50
108/218 [=============>................] - ETA: 2:35 - loss: 96.6736 - accuracy: 0.7268 - precision: 0.6762 - recall: 0.8673 - auc: 0.7947

KeyboardInterrupt: 

: 

In [ ]:

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
